In [1]:
import json
import requests
import gzip
from io import BytesIO

In [5]:
class RichCategoryIngestion:
    
    def __init__(self, gateway, grant_type, client_id, client_secret,api_key):
        self.gateway = gateway
        self.grant_type = grant_type
        self.client_id = client_id
        self.client_secret = client_secret
        self.api_key = api_key
    
    def post_authentication(self):
        
        url = f"https://{self.gateway}-gateway.richrelevance.com/streaming-ingest/v1/oauth2/token"
        data = {'grant_type':self.grant_type, 'client_id':self.client_id, 'client_secret':self.client_secret}
        header={'Content-Type':'application/x-www-form-urlencoded'}
        
        response = requests.post(url, params=data, headers = header)
        return json.loads(response.text)
    
    def zip_payload(self,payload: str) -> bytes:
        
        btsio = BytesIO()
        g = gzip.GzipFile(fileobj=btsio, mode='w')
        g.write(bytes(payload, 'utf8'))
        g.close()
        return btsio.getvalue()
    
    
    def put_create_category(self, snp_id: int, payload: dict):
        
        access_token = self.post_authentication()['access_token']
        url = f"https://{self.gateway}-gateway.richrelevance.com/streaming-ingest/v1/{self.api_key}/category?snapshotId={str(snp_id)}"
        header = {
                    'Authorization': f'Bearer {access_token}',
                    'Content-type' : 'application/json; charset=utf-8',
                    'Content-Encoding': 'gzip'
               }
        payload = self.zip_payload(payload)
        response = requests.patch(url, headers= header, data=payload)
        return json.loads(response.text)
        

In [16]:
brand = 'cyzone' #cyzone

In [17]:
file = f"D:\\belcorp-acoe-2020\\rich-relevance\\docs\\Data Model\\Data\\payloadRR_categories_{brand}.json"
env_credential = {"staging":{"gateway":"staging","api_key":"e1b75ced78fdfe64","client_id":"c5bac54c76907d4a","client_secret":"9jtqa8k0pcffrr72h755qce6ka"},"production":{"gateway":"gateway","api_key":"e1b75ced78fdfe64","client_id":"d987346d6347894b","client_secret":"uqlsl9q6emti9ketdaid98rkdm"}}
credentials = env_credential['staging']

In [18]:
rr_ingestion = RichCategoryIngestion(credentials['gateway'],'client_credentials',credentials['client_id'],credentials['client_secret'], credentials['api_key'])
with open(file, encoding="utf8") as json_file:
    file_list = []
    for line in json_file:
        category_dict = json.loads(line)
        category_dict[list(category_dict.keys())[0]]['overrides'] = {
            "language": {
                "es-PE":{
                    "properties":{
                        "name": category_dict[list(category_dict.keys())[0]]['properties']['name']
                    }
                },
                "es-MX":{
                    "properties":{
                        "name": category_dict[list(category_dict.keys())[0]]['properties']['name']
                    }
                },
                "es-CO":{
                    "properties":{
                        "name": category_dict[list(category_dict.keys())[0]]['properties']['name']
                    }
                },
                "es-PA":{
                    "properties":{
                        "name": category_dict[list(category_dict.keys())[0]]['properties']['name']
                    }
                },
                "es-CL":{
                    "properties":{
                        "name": category_dict[list(category_dict.keys())[0]]['properties']['name']
                    }
                },
            }
        }
        file_list.append(category_dict)
    
with open(f'payloadRR_categories_{brand}.json', 'w') as outputfile:
    json.dump(file_list, outputfile)
        #print(category_dict)
        #response = rr_ingestion.put_create_category(64, json.dumps(category_dict))
        #print(response)

In [ ]:
dict_pay = {'esika-40': {'properties': {'name': 'SETS'}, 'associations': {'category': {'parent': ['esika']}}, 'overrides': {'language': {'es-PE': {'properties': {'name': 'SETS'}}, 'es-MX': {'properties': {'name': 'SETS'}}, 'es-CO': {'properties': {'name': 'SETS'}}, 'es-PA': {'properties': {'name': 'SETS'}}, 'es-CL': {'properties': {'name': 'SETS'}}}}}}
response = rr_ingestion.put_create_category(60, json.dumps(dict_pay))
print(response)